<a href="https://colab.research.google.com/github/AMIT110409/intent-recognition-using-BERT-/blob/main/intent_classification_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import math
import datetime
from tqdm import tqdm



In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

## here tensorflow work as a backend

In [ ]:
!pip install bert
!pip install bert-for-tf2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3745 sha256=fd02d407e28e1563cd7f0aa5fa3a1ec233651d12bae18bcd259c58f657bbff94
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6777 sha256=e22551fda5696766899cb0c5c7d3f04285d793090e55bc98e9cb8cecbc52e506
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing m

In [ ]:
import bert
from bert import BertModelLayer ## use for building BERT modes for various task  .
from bert.loader import StockBertConfig,map_stock_config_to_params,load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer ## convert text to numerical


In [ ]:
pip show bert

Name: bert
Version: 2.2.0
Summary: BERT Serialization Library
Home-page: https://github.com/samuel/python-bert
Author: Samuel Stauffer
Author-email: samuel@descolada.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: erlastic
Required-by: 


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
random_seed = 30
np.random.seed(random_seed) #shuffling or sampling data
tf.random.set_seed(random_seed)  #weight initialization or dropout

## random seed used because it will produce same results each time  the program is run .

DATAset load


In [ ]:
import json
with open('data_full.json') as json_file:
  intent1 = json.load(json_file)
intent1_train = intent1['train']
intent1_test = intent1['test']
intent1_val = intent1['val']

In [ ]:
classes = ['insurance',
 'next_holiday',
 'repeat',
 'credit_limit_change',
 'book_hotel',
 'yes',
 'damaged_card',
 'rewards_balance',
 'time',
 'pto_balance',
 'interest_rate',
 'change_volume',
 'taxes',
 'sync_device',
 'traffic',
 'what_song',
 'shopping_list',
 'todo_list_update',
 'order_checks',
 'shopping_list_update']

In [ ]:
train_data=[]
test_data=[]
val_data=[]

In [ ]:
for i in intent1_train:
  if i[1] in classes:
    train_data.append(i)


In [ ]:
for i in intent1_test:
  if i[1] in classes:
    test_data.append(i)


In [ ]:
for i in intent1_val:
  if i[1] in classes:
    val_data.append(i)


In [ ]:
df = pd.DataFrame(train_data) ## making dataframe from json file
df.to_csv('train_data.csv',index=False,header=('text','intent'))
train = pd.read_csv('train_data.csv')
print(len(train))
train.head()


2000


,text,intent
0,"what time is it in punta gorda, florida",time
1,"what time is it in glenwood springs, co",time
2,"what time is it in fredericksburg, tx",time
3,"what time is it in las vegas, nv",time
4,"what time is it in houston, tx",time


In [ ]:
df = pd.DataFrame(val_data) ## here we are making dataframe for validation data
df.to_csv('val_data.csv',index=False,header=('text','intent'))
valid = pd.read_csv('val_data.csv')
valid.head()

,text,intent
0,what time is it in france,time
1,what's the time in london right now,time
2,what hour is it in london,time
3,what's the time,time
4,what is the time in london,time


In [ ]:
df = pd.DataFrame(test_data)
df.to_csv('test_data.csv',index=False,header=('text','intent'))
test = pd.read_csv('test_data.csv')
print(len(test))
test.head()

600


,text,intent
0,i need you to tell me what time it is in new y...,time
1,"what time is it in adelaide, australia right now",time
2,is it after noon,time
3,is it six o clock yet,time
4,please give me the time in tanzania at this mo...,time


In [ ]:
## here we append validation data with train data

train = train.append(valid).reset_index(drop=True)

<ipython-input-17-0cd1f85ee509>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(valid).reset_index(drop=True)


In [ ]:
os.makedirs("model",exist_ok=True) ## make directory name of model
bert_model_name = "uncased_L-12_H-768_A-12" #3 pretrained BERT model that we used here
bert_ckpt_dir = os.path.join("model/",bert_model_name)  ##create file path to the directory
bert_ckpt_file = os.path.join(bert_ckpt_dir,"bert_model.ckpt") ## creates a file path to the BERT model
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")##  CREATEs a file path to the berth model configuration file


Input Text Prepration

In [ ]:
class DataPreparation:

  text_column = "text"
  label_column = "intent"

  def __init__(self,train,test,tokenizer: FullTokenizer,classes,max_seq_len=192):
      self.tokenizer = tokenizer
      self.max_seq_len = 0
      self.classes = classes

      ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self.prepare_data,[train,test])

      print("max_seq_len",self.max_seq_len)
      self.max_seq_len = min(self.max_seq_len,max_seq_len)
      self.train_x, self.test_x = map(self.data_padding, [self.train_x, self.test_x])

  def prepare_Data(self,df):
    x,y = [] , []

    for _, row in tqdm(df.iterrows()):
      text,label = row[DataPreparation.text_column], row[DataPreparation.label_column]
      tokens = self.tokenizer.tokenize(text)
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
      self.max_seq_len = max(self.max_seq_len,len(token_ids))
      x.append(token_ids)
      y.append(self.classes.index(label))

    return np.array(x),np.array(y)

  def data_padding(self,ids):
    x = []
    for input_ids in ids:
        input_ids = input_ids[:min(len(input_ids),self.max_seq_len -2)]
        input_ids = input_ids + [0]*(self.max_seq_len - len(input_ids))
        x.append(np.array(input_ids))
    return np.array(x)









In [ ]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

NotFoundError: ignored

In [ ]:
##tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir,"vocab.txt"))

Model prepare

In [ ]:
def model_defination(max_seq_len,bert_ckpt_file):

    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None
      bert = BertModelLayer.from_params(bert_params,name="bert")

    input_ids = keras.layers.Input(shape=(max_seq_len,),dtype='int32',name="input_ids")
    bert_output = bert(input_ids)

    print("bert shape", bert_output.shape)

    cls_out = keras.layer.Lambda(lambda seq: seq[:,0,:])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768,activation="tanh")(cls_out)
    logits = keras.layer.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=len(classes),activation="softmax")(logits)

    model = keras.Model(input=input_ids, outputs=logits)
    model.build(input_shape=(None,max_seq_len))

    load_stock_weights(bert,bert_ckpt_file)

    return model






In [ ]:
classes = train.intent.unique().tolist()

data = DataPreparation(train,test,tokenizer,classes,max_seq_len=128)

NameError: ignored

In [ ]:
data.train_x.shape

In [ ]:
data.train_x[0]

In [ ]:
data.train_y[0]

In [ ]:
model = model_defination(data.max_seq_len,bert_ckpt_file)

In [ ]:
model.summary()

In [ ]:
model.compile(

    optimizer=keras.optimizers.Adam(1e-5)
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]

)

In [ ]:
log_dir = "log/intent_detection/" + datetime.dateetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[-3]
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
    x=data.train_x,
    y=data.train_y,
    validation_split=0.1,
    batch_size=16,
    shuffle=True,
    epochs=5,
    callbacks=[tensorboard_callback]

)



In [ ]:
train_acc = model.evaluate(data.train_x,data.train_y)
train_acc = model.evaluate(data.train_x,data.train_y)

Accuracy

In [ ]:
print("train_acc",train_acc)
print("test_acc",test_acc)

Now check the model **bold text**

In [ ]:
sentences = [
    "is it five o clock yet",
    "find me a house with good loacilty"
]
pred_tokens = map(tokenizer.tokenize, sentences)
pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"],pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
pred_token_ids = map(
    lambda tids: tids + [0]*(data.max_seq_len - len(tids)),pred_token_ids

)
pred_token_ids = np.array(list(pred_token_ids))
predictions = model.predict(pred_token_ids).argmax(axis=-1)
for text, label in zip(Sentences, predictions):
    print("text:",text, "\nintent:",classes[label])
    print()






